![](https://user-images.githubusercontent.com/83436724/166116197-cbe1a8bc-e1df-4cac-ad5e-9f6cd9f0f43c.png)


## Obtener datos desde diversas fuentes

#### Obtener datos desde APIs
#### Autor: Carlos Adrián Alarcón [LinkedIn](https://www.linkedin.com/in/carlos-adrian-alarcon-delgado/)


Muchos sitios web utilizan APIs para disponibilizar información. Una API es un servicio que permite la comunicación entre sistemas (ver [aquí](https://www.ibm.com/cloud/learn/api) mayor detalle técnico). Esta clase de servicios permiten establecer comunicación con un servidor y consultar los datos que están disponibles.

Típicamente, casi todas las instituciones públicas tienen disponibles APIs para consumir información. En algunos casos, es necesario leer la documentación para estar al tanto de cómo hacer la consulta o la estructura que tiene la información que se está trayendo. En muchos casos, la data viene en formato ``.json``. Para Python, leer esta clase de formatos no es un problema debido a que se asemejan a un conjunto de diccionarios o listas.

Existen otros servicios que permiten consumir APIs con diversos planes. Algunas web disponibilizan sus servicios con un servicio estándar gratuito, pero si se quiere más información, se tiene que pagar determinada cantidad de acuerdo al número de consultas que se hagan.

Para este ejercicio, utilizaremos el API de [Football Data](https://www.football-data.org/), que trae datos de las competencias de fútbol, las diferentes ligas, partidos y juadores. Para eso, tenemos que entrar a la página y crear un usuario. Esta API tiene una versión gratuita que es la que utilizaremos. Para comenzar a utilizar la API, necesitamos el api-token, que se encuentra en nuestro perfil una vez que ya tengamos el usuario creado asociado a la versión gratuita de la api


<img src="https://user-images.githubusercontent.com/83436724/166865690-b6e399f7-d3c1-40bf-beab-049d3ec68fad.png" alt="Drawing" style="width: 500px;"/>

Una vez que ya tengamos el api-token, hacemos la conexión haciendo uso de las librerías ``http.client`` y ``json``. Para traer datos a través de API's, tenemos que hacer uso del método ``GET``

In [9]:
## Cargar librerías
import http.client
import json

## Crear conexión
connection = http.client.HTTPConnection('api.football-data.org')

## Utilizar API-Token
headers = { 'X-Auth-Token': 'colocar aquí tu API TOKEN' }

## Usar método GET para traer datos
connection.request('GET', '/v2/players/44/matches', None, headers )

## Guardar resultado de consulta al API en formato .json
response = json.loads(connection.getresponse().read().decode())

## Imprimir el tipo de dato
print(type(response))

## Convertir en listas de Python y mostrar algunos valores
print(list(response.items())[0:4])


<class 'dict'>
[('count', 100), ('filters', {'permission': 'TIER_ONE', 'limit': 100}), ('player', {'id': 44, 'name': 'Cristiano Ronaldo', 'firstName': 'Cristiano Ronaldo', 'lastName': 'dos Santos Aveiro', 'dateOfBirth': '1985-02-05', 'countryOfBirth': 'Portugal', 'nationality': 'Portugal', 'position': 'Offence', 'shirtNumber': None, 'lastUpdated': '2021-10-13T08:04:10Z'}), ('matches', [{'id': 327016, 'competition': {'id': 2021, 'name': 'Premier League'}, 'season': {'id': 733, 'startDate': '2021-08-13', 'endDate': '2022-05-22', 'currentMatchday': 37}, 'utcDate': '2022-05-02T19:00:00Z', 'status': 'FINISHED', 'matchday': 35, 'stage': 'REGULAR_SEASON', 'group': None, 'lastUpdated': '2022-05-05T00:20:31Z', 'odds': {'msg': 'Activate Odds-Package in User-Panel to retrieve odds.'}, 'score': {'winner': 'HOME_TEAM', 'duration': 'REGULAR', 'fullTime': {'homeTeam': 3, 'awayTeam': 0}, 'halfTime': {'homeTeam': 1, 'awayTeam': 0}, 'extraTime': {'homeTeam': None, 'awayTeam': None}, 'penalties': {'homeT

Podemos ver la estructura del atributo ``matches`` en la consulta

In [14]:

response['matches']

[{'id': 327016,
  'competition': {'id': 2021, 'name': 'Premier League'},
  'season': {'id': 733,
   'startDate': '2021-08-13',
   'endDate': '2022-05-22',
   'currentMatchday': 37},
  'utcDate': '2022-05-02T19:00:00Z',
  'status': 'FINISHED',
  'matchday': 35,
  'stage': 'REGULAR_SEASON',
  'group': None,
  'lastUpdated': '2022-05-05T00:20:31Z',
  'odds': {'msg': 'Activate Odds-Package in User-Panel to retrieve odds.'},
  'score': {'winner': 'HOME_TEAM',
   'duration': 'REGULAR',
   'fullTime': {'homeTeam': 3, 'awayTeam': 0},
   'halfTime': {'homeTeam': 1, 'awayTeam': 0},
   'extraTime': {'homeTeam': None, 'awayTeam': None},
   'penalties': {'homeTeam': None, 'awayTeam': None}},
  'homeTeam': {'id': 66, 'name': 'Manchester United FC'},
  'awayTeam': {'id': 402, 'name': 'Brentford FC'},
  'referees': [{'id': 11488,
    'name': 'Simon Bennett',
    'role': 'ASSISTANT_REFEREE_N1',
    'nationality': 'England'},
   {'id': 11570,
    'name': 'Harry Lennard',
    'role': 'ASSISTANT_REFEREE_N

Una vez que hemos identificado la estructura de la consulta (esta también la podemos consultar en la documentación de la API [aquí](https://www.football-data.org/documentation/quickstart)), procedemos a crear un ciclo para poner todos los datos en un formato entendible. Para este caso, utilizaremos un DataFrame de ``Pandas``

In [11]:
## Importar la librería
import pandas as pd

## Generar ciclo y guardar todo en una lista de Python
registrosTotales = []

for i in range(0,len(response['matches'])):
    idMatch = response['matches'][i]['id']
    nombreJugador = response['player']['firstName']
    apellidoJugador = response['player']['lastName']
    competition = response['matches'][i]['competition']['name']
    local = response['matches'][i]['homeTeam']['name']
    visitante = response['matches'][i]['awayTeam']['name']
    fechaPartido = response['matches'][i]['utcDate']
    golLocalPrimerTiempo =  response['matches'][i]['score']['halfTime']['homeTeam']
    golLocalFinal =  response['matches'][i]['score']['fullTime']['homeTeam']
    golVisitaPrimerTiempo =  response['matches'][i]['score']['halfTime']['awayTeam']
    golVisitaFinal =  response['matches'][i]['score']['fullTime']['awayTeam']
    arbitroPrincipal = response['matches'][i]['referees'][0]['name']
    registro = [idMatch,nombreJugador,apellidoJugador,competition,local,visitante,fechaPartido,golLocalPrimerTiempo,golVisitaPrimerTiempo,golLocalFinal,golVisitaFinal,arbitroPrincipal]
    registrosTotales.append(registro)

## Convertir la lista en un DataFrame de Pandas
detalleJugador = pd.DataFrame(registrosTotales, columns = ['idMatch','nombreJugador','apellidoJugador','competencia','equipoLocal','equipoVisita','fechaPartido','golLocalPrimerTiempo','golVisitaPrimerTiempo','golLocalFinal','golVisitaFinal','arbitroPrincipal'])

## Mostrar los primeros 05 registros del DataFrame
detalleJugador.head()



,idMatch,nombreJugador,apellidoJugador,competencia,equipoLocal,equipoVisita,fechaPartido,golLocalPrimerTiempo,golVisitaPrimerTiempo,golLocalFinal,golVisitaFinal,arbitroPrincipal
0,327016,Cristiano Ronaldo,dos Santos Aveiro,Premier League,Manchester United FC,Brentford FC,2022-05-02T19:00:00Z,1,0,3,0,Simon Bennett
1,326996,Cristiano Ronaldo,dos Santos Aveiro,Premier League,Manchester United FC,Chelsea FC,2022-04-28T18:45:00Z,0,0,1,1,Ian Hussin
2,327023,Cristiano Ronaldo,dos Santos Aveiro,Premier League,Arsenal FC,Manchester United FC,2022-04-23T11:30:00Z,2,1,3,1,Craig Pawson
3,327036,Cristiano Ronaldo,dos Santos Aveiro,Premier League,Manchester United FC,Norwich City FC,2022-04-16T14:00:00Z,2,1,3,2,Andy Madley
4,327046,Cristiano Ronaldo,dos Santos Aveiro,Premier League,Everton FC,Manchester United FC,2022-04-09T11:30:00Z,1,0,1,0,Marc Perry
